# TensorFlow Digit Recognizer using Built-in RNN functions

In [1]:
import tensorflow as tf 
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
element_size = 28
time_steps = 28
num_classes = 10
hidden_layer_size = 128
batch_size = 128

In [4]:
_inputs = tf.placeholder(tf.float32, [None, time_steps, element_size], name="inputs")
y = tf.placeholder(tf.float32, [None, num_classes], name="inputs")

In [5]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

In [6]:
Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=0.01))
bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=0.01))

In [7]:
def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl
    
last_rnn_output = outputs[:,-1, :]

final_output = get_linear_layer(last_rnn_output)

In [9]:
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=y))

optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy_loss)

correct_predictions = tf.equal(tf.argmax(y, 1), tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_predictions, tf.float32)))*100

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_labels = mnist.test.labels[:batch_size]

for i in range(10000):
    
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, element_size))
    
    sess.run(optimizer, feed_dict={
        _inputs: batch_x, y: batch_y
    })
    
    if i%1000 == 0:
        acc, loss = sess.run([accuracy, cross_entropy_loss], feed_dict={
        _inputs: batch_x, y: batch_y
        })
        print "Iteration: " + str(i) +", Minibatch loss: {:.6f}".format(loss)\
        + ", Training Accuracy: {:.5f}".format(acc)
    
    
test_acc, test_loss = sess.run([accuracy, cross_entropy_loss], feed_dict={
        _inputs: test_data, y: test_labels
})

print "Test Accuracy: {:.5f}".format(test_acc) + ", Test loss: {:.6f}".format(test_loss)


Iteration: 0, Minibatch loss: 2.299481, Training Accuracy: 12.50000
Iteration: 1000, Minibatch loss: 0.128221, Training Accuracy: 96.87500
Iteration: 2000, Minibatch loss: 0.094989, Training Accuracy: 96.09375
Iteration: 3000, Minibatch loss: 0.149405, Training Accuracy: 97.65625
Iteration: 4000, Minibatch loss: 0.032887, Training Accuracy: 99.21875
Iteration: 5000, Minibatch loss: 0.063167, Training Accuracy: 98.43750
Iteration: 6000, Minibatch loss: 0.006450, Training Accuracy: 100.00000
Iteration: 7000, Minibatch loss: 0.009558, Training Accuracy: 100.00000
Iteration: 8000, Minibatch loss: 0.021926, Training Accuracy: 99.21875
Iteration: 9000, Minibatch loss: 0.011684, Training Accuracy: 100.00000
Test Accuracy: 96.87500, Test loss: 0.127121
